# Exercise 4: Biostatistical analysis

Our previous survival analyses highlighted that the drugB could have a limited effect on several subgroups of the population (ex1). Therefore, clinicians have asked us to dig further the analysis on this medication and try to quantify its effect on death event, along with other clinical and biological features. To that end, we will implement Multiple Linear Regression models, which allow to "explain" an event (e.g death) by a pre-defined set of quantitative and qualitative features.

A restricted view of the database has been constructed to remove information about the patients prescribed with drugA, so that we can focus on drugB vs control population.
Let's assume for this exercise that this view has also been cleaned up by a previous pre-processing work (cf ex 1,2,3). We will focus on merging the different tables available into one that we will use to perform our statistical analyses.

The following libraries are required to go through this notebook:

In [2]:
import pandas as pd
import numpy as np

# For descriptive statistics
from scipy.stats import ttest_ind, chi2_contingency

# For Logistic Models implementation
from statsmodels.formula.api import logit, glm

# Table of content

1. [Preprocessing](#preprocessing)  
    1.1 [Age](#age)  
    1.2 [Gender](#gender)  
    1.3 [Death](#death)  
    1.4 [Comorbidities](#comorb)  
    1.5 [Biological values](#bio)  
    1.6 [Medication data](#med)
2. [Descriptive statistics](#stat)  
3. [Statistical analysis](#analysis)  
    3.1 [Univariate analaysis](#uni_analysis)  
    3.2 [Generalized Linear Model](#multi_analysis)  
4. [Takeaways](#takeaways)


<a id="preprocessing"></a>
# 1. Preprocessing

The main objective of this step is to prepare a ready-to-use table for the subsequent statistical analyses. We want to summarize the demographic, clinical and biological information for each patient of the cohort.

Hence, the output DataFrame will include one row by patient, having the following columns:
- age at the time of the visit (*int*)
- male (*bool*)
- death at the time of the study (*bool*). We assume that the study is realized long enough after the visits so that we can overlook censoring issues.
- comorbidities : cancer antecedent and diabetes (*bool*)
- biological profile at the time of admission (*float*). The following items were measured for each patient : bmi, urea, hemoglobin, crp.
- drubB taken (*bool*)

**NB: For the sake of simplicity, we assume that all the available tables have been pre-processed (deduplication, NLP enhancing , data cleaning, ...) and can be used as is. Moreover, we assume that each patient has one and only visit associated.**

Open the following files using the `pandas.read_pickle()` function : 
  - *data/df_person.pkl* as `df_person`
  - *data/df_visit.pkl* as `df_visit`
  - *data/df_condition.pkl* as `df_cond`
  - *data/df_med.pkl* as `df_med`
  - *data/df_bio.pkl* as `df_bio.pkl`

In [3]:
# Patients
df_person = pd.read_pickle('data/df_person.pkl')
# Visits
df_visit = pd.read_pickle('data/df_visit.pkl')
# Diagnosis (condition)
df_condition = pd.read_pickle('data/df_condition.pkl')
# Medication
df_med = pd.read_pickle('data/df_med.pkl')
# Biology
df_bio = pd.read_pickle('data/df_bio.pkl')

Create the base output dataset (format : `pandas.DataFrame` ; name : `data`)  from the `df_person` table, gathering patient id, gender, birth and death dates.

In [4]:
data = df_person[#TODO]

Check the size of the dataframe using `dataframe.shape()`

In [5]:
data.shape

(20000, 4)

<a id="age"></a>
## 1.1 Age

Compute the patient age at the time of their hospitalization.

TIPS : 
- Inner join the `data` dataset and the `df_visit` dataset on `person_id` using the `pandas.merge()` function (see [doc](https://pandas.pydata.org/docs/reference/api/pandas.merge.html))
- Create an `age` column compute the age for each patient.

In [6]:
data = pd.merge(data, #TODO, on="person_id", how="inner")

In [7]:
data['age'] = #TODO

<a id="gender"></a>
## 1.2 Gender

Create a "male" boolean column to identify if the patient is a male (1) or a female (0).

In [8]:
data["male"] = #TODO

<a id="death"></a>
## 1.3 Death

Create a "dead" boolean column to identify if the patient has died or not.

TIP : The patient is dead if a date of death is mention. Use the function `pandas.notna()`

In [9]:
data["dead"] = #TODO

<a id="comorb"></a>
## 1.4 Comorbidities : cancer and diabetes antecedent

Gathering with experts lead us to identify the ICD-10 codes corresponding to cancer antecedent and diabetes :
- cancer antecedent : ["Z851", "Z852", "Z853"]
- diabetes : ["E10", "E11", "E12"]

Create a `comorbidity` column by tagging each code in the `df_cond` Dataframe corresponding to diabetes or cancer antecedent.

TIP: you can build a `code_to_comorb` dictionary mapping each ICD-10 code to its corresponding comorbidity, and use the pandas `df.col_name.map(dict)` built-function to map the values of the `condition_source_value` column.

In [10]:
comorb_to_code = {
    "diabetes": #TODO,
    "atcd_cancer": #TODO
}

In [11]:
code_to_comorb = {}
for key,codes_list in comorb_to_code.items():
    for code in codes_list:
        code_to_comorb[code] = #TODO

In [12]:
df_condition["comorbidity"] = #TODO

Create a `df_comorbidities` DataFrame by selecting the condition codes corresponding to the desired comorbidities.

TIP : 
You can use the `df.col_name.notna()` built-in function : 
```python 
df[df.col_name.notna()]
```

In [13]:
df_comorbidities = #TODO

How many patient have cancer antecedent ? How many have had diabetes ?

TIP : Use the `df.col_name.value_counts()` function

In [14]:
df_comorbidities.comorbidity.value_counts()

diabetes       3213
atcd_cancer     664
Name: comorbidity, dtype: int64

The `df_comorbidities` DataFrame is composed of one row per condition source value (either cancer or diabete).
<br> A patient can have multiple rows depending on how many conditions they have.

Now, we will create a dataframe with one row per patient and per visit : 
<br>NB : each patient only has one and only visit

| visit_occurrence_id | person_id  | atcd_cancer | diabetes |
| :------------------ | :----------|:------------| :---------- |
| aaa | xxx | True |  True |
| bbb | yyy | False |  True |
| ccc | zzz | False |  False |

Pivot the `df_comorbidities` DataFrame to have the presence of both antecedents for each patient. Don't forget to replace the NaN values by False and to reset the index !

TIP 
You can use the [pivot](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.pivot.html) built-in function :
```python
df.pivot(index=["index_col"], columns=["col"], values=["value"]])
```
You might need to create "artificially" the *value* column as a True boolean, and drop the *condition_occurrence_id* column.

In [15]:
df_comorbidities["value"] = True

c:\Users\Charline\miniconda3\envs\eds-tuto\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [16]:
df_comorbidities = df_comorbidities.drop(columns=["condition_occurrence_id"]).pivot(index=#TODO, columns=#TODO, values="value").reset_index().fillna(#TODO)

Merge the `data` DataFrame with the obtained `df_comorbidities` to summarize comorbidities information for each patient.

TIP:  
- Think twice about the type of merging you want to perform
- Do not forget to fill NA values for the comorbidity columns with False !

In [17]:
data = pd.merge(data, df_comorbidities[["person_id", "atcd_cancer", "diabetes"]], on=#TODO, how=#TODO).fillna(#TODO)

<a id="bio"></a>
## 1.5 Biological profile

Check the different biological values available.

In [18]:
#TODO

urea    20000
bmi     20000
hb      20000
crp     20000
Name: concept_source_value, dtype: int64

Create a `df_bio_processed` DataFrame with all biological values for each visit : 

| visit_occurrence_id | bio_A  | bio_B | bio_C |
| :------------------ | :----------|:------------| :---------- |
| aaa | 189 | 13 |  0.87 |
| bbb | 160 | 11 |  0.59 |
| ccc | 209 | 11 |  0.69 |


TIP : Pivot the `df_bio` DataFrame and retrieve the biological values measured at the admission of each hospitalization.

In [19]:
bio_processed = df_bio[["visit_occurrence_id", "concept_source_value", "transformed_value"]].pivot(index=#TODO, columns=#TODO, values="transformed_value").reset_index()

Merge `data` and `bio_processed` on `visit_occurrence_id`

TIP: Think twice about the type of merging you want to perform

In [20]:
data = #TODO

<a id="med"></a>
## 1.6 Medication taken

Create a "drugB" boolean column in `data` indicating whether patients have taken the drugB or not. 

TIP :
You can pivot the `df_med` DataFrame into a `med_processed` dataframe as done for the comorbidities and merge it with `data`.

In [21]:
df_med["value"]=True

In [22]:
med_processed = df_med[["visit_occurrence_id", "drug_source_value", "value"]].pivot(#TODO).reset_index().fillna(#TODO)

In [23]:
data = #TODO

### Final step

Select the columns you want to keep for the statistical analyses and check that you don't have missing values.

TIP : you can use the following built-in function :
```python
df.isna().sum()
```
Beware of the axis along which you sum !

In [24]:
data = data[["person_id", "age", "male", "dead", "atcd_cancer", "diabetes", "bmi", "crp", "urea", "hb", "drugB"]]

In [25]:
#TODO

person_id      0
age            0
male           0
dead           0
atcd_cancer    0
diabetes       0
bmi            0
crp            0
urea           0
hb             0
drugB          0
dtype: int64

<a id="stat"></a>
# 2. Descriptive statistics

Let's compute base statistics for our features in order to catch a glimpse of our population characteristics by separating dead vs alive patients. We will compute :
- For quantitative features : mean (std) and student t-test
- For qualitative features : % of prevalence and Chi-sq test

*Assuming that we are respecting the initial conditions of each test (iid gaussian distribution for quanti feats).*

We are going to use the `df.groupby().agg(agg_dict)` function. Check out the doc [here](https://pandas.pydata.org/pandas-docs/version/0.22/generated/pandas.core.groupby.DataFrameGroupBy.agg.html)  
We will therefore need to provide for each column of `data` the desired aggregation function.

Create a "prevalence" function that returns the percentage of True values from a list of booleans.

In [26]:
def prevalence(values):
    return #TODO

Create the "agg_dict" dictionary storing for each column name (keys) the list of transforming functions (values). You can use the `np.mean` and `np.std` registered functions.

TIP : Example
```python
agg_dict = {
    "person_id": ["count"],
    "age": [np.mean, np.std],
    ...
}
```

In [27]:
agg_dict = {
    "person_id": #TODO,
    "age": #TODO,
    "male": #TODO,
    "atcd_cancer" : #TODO,
    "diabetes" : #TODO,
    "drugB": #TODO,
    "bmi" : #TODO,
    "crp": #TODO,
    "hb": #TODO,
    "urea": #TODO,
}

Create a summary DataFrame from `data` using the predefined `agg_dict` to compare base statistics between the living and the dead groups.

In [28]:
data_summary = round(data.groupby(#TODO).agg(#TODO).T,2)

In [29]:
data_summary

dead                      False     True 
person_id   count       8930.00  11070.00
age         mean          39.74     58.15
            std           28.55     26.50
male        prevalence    49.81     50.15
atcd_cancer prevalence     2.95      3.62
diabetes    prevalence    13.17     18.40
drugB       prevalence    68.57     44.96
bmi         mean          22.22     22.96
            std            3.83      3.27
crp         mean           4.77      4.71
            std            0.37      0.36
hb          mean          13.24     13.47
            std            1.62      1.59
urea        mean           1.86      2.01
            std            0.44      0.38

Compute statistical tests to assess the significancy of the resulting difference between the dead and the living groups.  
For numerical values (age and bio), you can compute the p-values associated to Student t-tests (function `ttest_ind`, doc [here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html)), and for categorical values (gender, comorbidities and drug) chi-sq tests (function `chi2_contigency`, doc [here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.chi2_contingency.html)).

TIP : 
You may need to separate `data` into `data_alive` and `data_dead` in order to facilitate the statistic computation.

In [30]:
numerical = ["age", "bmi", "crp", "hb", "urea"]
test_categorical = ["male", "atcd_cancer", "diabetes", "drugB"]

In [31]:
data_alive = data.query("dead==False")
data_dead = data.query("dead==True")

In [32]:
for feat, sub_value in data_summary.index:
    if sub_value == 'std':
        continue
    if feat in numerical:
        data_summary.loc[(feat,sub_value),"t-test"] = round(ttest_ind(data_alive[feat], data_dead[feat])[1],4)

    if feat in test_categorical:
        data_summary.loc[(feat,sub_value),"chi-sq"] = round(chi2_contingency(pd.crosstab(data["dead"], data[feat]))[1],4)


In [33]:
data_summary

dead                      False      True  t-test  chi-sq
person_id   count       8930.00  11070.00     NaN     NaN
age         mean          39.74     58.15     0.0     NaN
            std           28.55     26.50     NaN     NaN
male        prevalence    49.81     50.15     NaN  0.6388
atcd_cancer prevalence     2.95      3.62     NaN  0.0088
diabetes    prevalence    13.17     18.40     NaN  0.0000
drugB       prevalence    68.57     44.96     NaN  0.0000
bmi         mean          22.22     22.96     0.0     NaN
            std            3.83      3.27     NaN     NaN
crp         mean           4.77      4.71     0.0     NaN
            std            0.37      0.36     NaN     NaN
hb          mean          13.24     13.47     0.0     NaN
            std            1.62      1.59     NaN     NaN
urea        mean           1.86      2.01     0.0     NaN
            std            0.44      0.38     NaN     NaN

What can you conclude ?

Dead patients appear to be older, having diabetes more frequently and presenting a worse biological profile (especially regarding Urea) at admission.
**However**, the statistical significancy must be confronted to the sample size. Indeed, with >20000 patients, the statistical tests are de facto significant, and those p-values are "artificially" low.

<a id="analysis"></a>
# 3. Statistical analysis

Now that we have a better overview of our cohort and its characteristics, we can perform the desired statistical analysis by fitting logistic regression models.

Convert binary features into integer, as boolean are not supported by the statsmodels functions.

In [34]:
categorical = ["dead","male", "atcd_cancer", "diabetes", "drugB"]
data[categorical] = data[categorical].astype(int)

<a id="uni_analysis"></a>
## 3.2 Univariate logistical regression : OR associated with taking drugB

Build a logistical regression model to evaluate the univariate effect of taking drug B on death, and fit it on `data`. Documentation about the `logit` function can be found [here](https://www.statsmodels.org/devel/generated/statsmodels.formula.api.logit.html).

In [35]:
model = logit(#TODO).fit(disp=False)
print(model.summary())

                           Logit Regression Results                           
Dep. Variable:                   dead   No. Observations:                20000
Model:                          Logit   Df Residuals:                    19998
Method:                           MLE   Df Model:                            1
Date:                Wed, 08 Jun 2022   Pseudo R-squ.:                 0.04115
Time:                        17:47:17   Log-Likelihood:                -13182.
converged:                       True   LL-Null:                       -13748.
Covariance Type:            nonrobust   LLR p-value:                4.631e-248
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.7750      0.023     33.975      0.000       0.730       0.820
drugB         -0.9822      0.030    -33.025      0.000      -1.041      -0.924


Extract the coefficient associated with drugB using `model.params[param_name]`

In [36]:
#TODO

-0.9822491465060246

Compute the Odds Ratio associated with the drugB feature.

TIP :
OR = exp(feature_coef)

In [37]:
or_uni = #TODO

What can you conclude of the risk of death when taking drugB ?

In [38]:
print(f"According to our univariate analysis, the risk of dying appears to be multiplied by {#TODO}, or reduced by {#TODO} when taking drug B.")

According to our univariate analysis, the risk of dying appears to be multiplied by 0.37, or reduced by 2.67 when taking drug B.


Perform the same univariate analysis on the other features relating to death event.

In [39]:
for feat in ["age", "male", "atcd_cancer", "diabetes", "urea", "crp", "bmi", "hb"]:
    model = logit(f"dead ~  {#TODO}", #TODO).fit(disp=False)
    print(f"Univariate OR associated with {feat}: {#TODO}")


Univariate OR associated with age: 1.02
Univariate OR associated with male: 1.01
Univariate OR associated with atcd_cancer: 1.24
Univariate OR associated with diabetes: 1.49
Univariate OR associated with urea: 2.51
Univariate OR associated with crp: 0.64
Univariate OR associated with bmi: 1.06
Univariate OR associated with hb: 1.09


What parameters seem to have the greatest influence on the death event ?

The most influent parameters on death are the following :
#TODO

<a id="multi_analysis"></a>
## 3.3 Multivariate analysis : Generalized Linear Models

Build a Generalized Linear Model to explain the death event with all the available features. Documentation about the `glm` function can be found [here](https://www.statsmodels.org/devel/generated/statsmodels.formula.api.glm.html).

In [40]:
model = glm(#TODO).fit()
print(model.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                   dead   No. Observations:                20000
Model:                            GLM   Df Residuals:                    19990
Model Family:                Gaussian   Df Model:                            9
Link Function:               identity   Scale:                         0.21239
Method:                          IRLS   Log-Likelihood:                -12880.
Date:                Wed, 08 Jun 2022   Deviance:                       4245.6
Time:                        17:47:18   Pearson chi2:                 4.25e+03
No. Iterations:                     3   Pseudo R-squ. (CS):             0.1514
Covariance Type:            nonrobust                                         
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       0.0522      0.059      0.885      

Compute the stratified OR associated with taking drugB :

In [41]:
or_multi = #TODO

Compute the OR associated with each feature after adjustement.

In [42]:
for feat, param in model.params.to_dict().items():
    print(feat, #TODO)

Intercept 1.05
age 1.0
male 0.98
atcd_cancer 1.0
diabetes 1.02
drugB 0.8
bmi 1.01
crp 1.02
hb 1.0
urea 1.04


What can you conclude ?

In [43]:
print(f"After a stratification on other features, the risk of dying appears to be reduced by  {#TODO} when taking drug B.")

After a stratification on other features, the risk of dying appears to be reduced by  1.25 when taking drug B.


The most impactful features on the adjusted model is:
- #TODO

**CONCLUSION**
The effect of drug B adjusted on all available features is reduced, illustrating the fact that this medication was more frequently prescribed to younger people in better global health (less comorbidities and better biological features). 
Moreover, the effect of the other features (urea, comorbidities, etc..) vanishes when adjusting on drugB.  
This is a typical clinical bias that can be found frequently in Real-World Data and may lead to over-interpretation of drug effects.. do not conclude to quickly !

# 4. Takeaways

Large scale Real-World Data may be subject to major clinical biases, such as prescription bias. They can affect the overall population, or be subject to differing guidelines and practices among the diversity of hospitals, clinicians and patients. 
It is of paramount importance to work closely with clinician to be aware of such biases and avoid over-interpretation, as long as performing stratified analyses to have a broader view of the effects.